# 使用 ADK + 向量搜尋建立電子商務推薦 AI 代理程式

原作者：[Kaz Sato](https://github.com/kazunori279)

在本教學中，我們將探討如何為電子商務網站建立一個簡單的多代理程式系統，旨在提供您在 [Shopper's Concierge 示範](https://www.youtube.com/watch?v=LwHPYyw7u6U) 中找到的「生成式推薦」。此處也提供逐步指南影片：[此處](https://youtu.be/07GX28rk7Yc)。

### 什麼是生成式推薦？

生成式推薦指的是 AI 不僅能擷取直接符合使用者明確搜尋查詢的項目，還能**根據對使用者意圖、外部研究或上下文資訊的更深入理解，智慧地推斷、擴展或建立新的搜尋查詢和項目建議。**

以下是其展示的功能細分：

1.  **理解和擴展使用者意圖：** 系統不僅僅是接收直接的查詢，而是能夠解讀潛在的需求。例如，當使用者要求「給 10 歲男孩的生日禮物」時，AI 不僅僅是搜尋這些確切的關鍵字。
2.  **利用外部研究 (Google 搜尋)：** 「生成式推薦」過程涉及使用像 Google 搜尋這樣的工具進行市場研究。這使得 AI 能夠理解對於給定的意圖，通常會購買哪種類型的商品 (例如，10 歲男孩的熱門禮物)。
3.  **產生新的查詢：** 基於這項研究，AI 會主動*產生*一個更具體和多樣化的搜尋查詢列表。這些產生的查詢超越了最初的使用者輸入，旨在擴大搜尋範圍並找到更相關的結果 (例如，「10 歲兒童的教育玩具」、「男孩的冒險書籍」、「兒童的編碼套件」)。

從本質上講，「生成式推薦」超越了簡單的關鍵字比對，採用了一種更動態、更智慧的方法，AI 會主動協助使用者產生新的想法和相關的搜尋路徑，幫助他們發現想要的產品。

最後，我們將建立一個依循以下流程運作的代理程式系統：

![商店代理程式序列圖](../../../docs/assets/shop_agent.png)

<!-- 圖表的 Mermaid 程式碼
sequenceDiagram
   participant User as 使用者
   participant Shop Agent as 商店代理程式
   participant Research Agent as 研究代理程式
   participant Google Search as Google 搜尋
   participant find_shopping_items as 尋找購物項目
   participant Vector Search as 向量搜尋

   User->>Shop Agent: 有沒有給我兒子的生日禮物？
   activate Shop Agent
   Shop Agent->>Research Agent: 有沒有給我兒子的生日禮物？
   activate Research Agent
   Research Agent->>Google Search: 搜尋男孩的生日禮物
   activate Google Search
   Google Search->>Research Agent: 搜尋結果
   deactivate Google Search
   Research Agent->>Research Agent: 產生 5 個查詢
   Research Agent->>Shop Agent: 「STEM 玩具」、「樂高積木組」...
   deactivate Research Agent
   Shop Agent->>User: 這裡有建議的查詢：「STEM 玩具」、「樂高積木組」...
   Shop Agent->>User: 用這些查詢來搜尋可以嗎？
   deactivate Shop Agent

   activate Shop Agent
   User->>Shop Agent: 好的，請！
   activate find_shopping_items
   Shop Agent->>find_shopping_items: 「STEM 玩具」、「樂高積木組」...
   activate Vector Search
   find_shopping_items->>Vector Search: 「STEM 玩具」、「樂高積木組」...
   Vector Search->>find_shopping_items: 「兒童對講機」、「Minecraft 探險家包」...
   deactivate Vector Search
   find_shopping_items->>Shop Agent : 「兒童對講機」、「Minecraft 探險家包」...
   deactivate find_shopping_items
   Shop Agent->>User : 「兒童對講機」、「Minecraft 探險家包」...
   deactivate Shop Agent
-->

## 安裝 ADK

首先，我們將安裝 ADK。在 Colab Enterprise 中，您可能會看到 `ERROR: pip's dependency resolver does...`，但您可以忽略它。

In [46]:
%pip install --upgrade google-adk==1.4.2 -q

### 匯入函式庫

In [ ]:
# 匯入必要的函式庫
import os
import logging
import asyncio
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import  InMemorySessionService
from google.adk.tools.agent_tool import AgentTool
from google.genai import types

# 忽略來自 ADK 和 Gemini API 的警告
logging.getLogger("google.adk.runners").setLevel(logging.ERROR)
logging.getLogger("google_genai.types").setLevel(logging.ERROR)

### 設定環境變數

從 [Google AI Studio](https://aistudio.google.com/apikey) 取得 API 金鑰，將金鑰設定到以下的 `GOOGLE_API_KEY`，然後執行儲存格以設定執行 ADK 所需的環境變數。

In [ ]:
from getpass import getpass

# 設定執行 ADK 所需的環境變數 (使用 Gemini API 金鑰)
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "False"
os.environ["GOOGLE_API_KEY"] = getpass("輸入您的 Gemini API 金鑰：")

# 若要在 Colab Enterprise 或 Cloud Workbench 中改用 Vertex AI，請使用以下程式碼：
#[PROJECT_ID] = !gcloud config list --format "value(core.project)"
#os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
#os.environ["GOOGLE_CLOUD_LOCATION"] = "us-central1"
#os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "True"

## 定義 **test_agent** 函式以測試代理程式

為了測試我們將要建構的代理程式，我們需要定義一個函式 `test_agent`，它使用 `Runner` 和 `SessionService` 來模擬一個代理程式執行階段環境。若要深入了解代理程式執行階段，請參閱 [代理程式執行階段](https://google.github.io/adk-docs/runtime/) 文件。

In [20]:
from google.genai import types

# 定義用於測試代理程式的 app_name、user_id 和 session_id
APP_NAME = "shop_concierge_app"
USER_ID = "user_1"

session_service = InMemorySessionService()

async def test_agent(query, agent):
  """向代理程式傳送查詢並列印最終回應。"""

  print(f"\n>>> 使用者查詢：{query}")

  # 建立一個會話
  session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
  )

  # 建立一個 Runner
  runner = Runner(
      app_name=APP_NAME,
      agent=agent,
      session_service=session_service,
  )

  # 以 ADK 格式準備使用者訊息
  content = types.Content(role='user', parts=[types.Part(text=query)])

  final_response_text = None
  # 我們迭代 run_async 的事件以尋找最終答案。
  async for event in runner.run_async(user_id=USER_ID, session_id=session.id, new_message=content):
      if event.is_final_response():
          if event.content and event.content.parts:
             final_response_text = event.content.parts[0].text
          break
  print(f"<<< 代理程式回應：{final_response_text}")

## 定義一個 **商店代理程式**

讓我們定義一個商店代理程式並測試它。請注意，此代理程式目前沒有任何搜尋功能。

In [ ]:
instruction = f'''
    您的角色是電子商務網站上的商店搜尋代理程式，該網站擁有數百萬件商品。您的職責是根據使用者查詢搜尋商品。
'''

shop_agent = Agent(
    model='gemini-2.5-flash',
    name='shop_agent',
    description=(
        '電子商務網站的商店代理程式'
    ),
    instruction=instruction,
)

In [22]:
await test_agent("這是什麼樣的網站？", shop_agent)


>>> 使用者查詢：這是什麼樣的網站？
<<< 代理程式回應：我是一個電子商務網站的商店代理程式，擁有數百萬件商品。



## 定義 **call_vector_search** 以呼叫向量搜尋後端

對於上述的基本代理程式，我們希望新增一個項目搜尋功能。為此，我們在此定義一個函式 `call_query_api`，它會向 [向量搜尋互動式示範](https://cloud.google.com/vertex-ai/docs/vector-search/try-it) 提供的 REST 端點傳送一個 HTTP 請求。有關傳送到端點的每個參數的詳細資訊，請參閱示範頁面。

In [23]:
import requests
import json

def call_vector_search(url, query, rows=None):
    """
    呼叫向量搜尋後端進行查詢。

    Args:
        url (str): 搜尋端點的 URL。
        query (str): 查詢字串。
        rows (int, optional): 要傳回的結果列數。預設為 None。

    Returns:
        dict: 來自 API 的 JSON 回應。
    """

    # 建立 HTTP 標頭和負載
    headers = {'Content-Type': 'application/json'}
    payload = {
        "query": query,
        "rows": rows,
        "dataset_id": "mercari3m_mm", # 使用 Mercari 3M 多模態索引
        "use_dense": True, # 使用多模態搜尋
        "use_sparse": True, # 也使用關鍵字搜尋
        "rrf_alpha": 0.5, # 兩個結果以相同的權重合併
        "use_rerank": True, # 使用 Ranking API 進行重新排名
    }

    # 向搜尋端點傳送 HTTP 請求
    try:
        response = requests.post(url, headers=headers, data=json.dumps(payload))
        response.raise_for_status()  # 對不好的狀態碼引發例外
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"呼叫 API 時發生錯誤：{e}")
        return None

## 定義 **find_shopping_items** 工具

現在，我們將使用名為 `find_shopping_items` 的 ADK 工具來包裝 `call_vector_search` 函式。請注意，我們需要 1) 使用明確的類型，例如 `queries: list[str]`，以及 2) 使用詳細的文件字串，這兩者都是為了向代理程式傳達此工具的功能和語意。

有關 ADK 工具機制的詳細資訊，請參閱 ADK 文件中的[工具](https://google.github.io/adk-docs/tools/)。

In [24]:
from typing import Dict

def find_shopping_items(queries: list[str]) -> Dict[str, str]:
    """
    使用指定的查詢列表從電子商務網站尋找購物項目。

    Args:
        queries: 要執行的查詢列表。
    Returns:
        一個具有以下一個屬性的字典：
            - "status": 傳回以下狀態：
                - "success": 成功執行
            - "items": 在電子商務網站上找到的項目。
    """
    url = "https://www.ac0.cloudadvocacyorg.joonix.net/api/query"

    items = []
    for query in queries:
        result = call_vector_search(
            url=url,
            query=query,
            rows=3,
        )
        items.extend(result["items"])

    print("-----")
    print(f"使用者查詢：{queries}")
    print(f"找到：{len(items)} 個項目")
    print("-----")

    return items

讓我們來測試這個工具。

In [25]:
find_shopping_items(["有跳舞的人的杯子", "有跳舞的動物的杯子"])

-----
使用者查詢：['有跳舞的人的杯子', '有跳舞的動物的杯子']
找到：6 個項目
-----


[{'dense_dist': 0.0,
  'description': 'Free People 女士日落跳舞洋裝粉紅色 M 號\n\n品牌：Free People \n款式：日落跳舞洋裝\n顏色：粉紅色\n尺寸：M\nSku 編號：9968\n狀況：全新無吊牌\n串珠繫帶細節\n腰部有梯形蕾絲\n全內襯',
  'id': 'm47216877271',
  'img_url': 'https://u-mercari-images.mercdn.net/photos/m47216877271_1.jpg?w=200&h=200&fitcrop&sharpen',
  'name': 'Free People 女士日落跳舞洋裝粉紅色 M 號',
  'rerank_score': 0.0,
  'sparse_dist': 0.4827784597873688,
  'url': 'https://www.mercari.com/us/item/m47216877271'},
 {'dense_dist': 0.20211432874202728,
  'description': '復古 hardanger 舞者 bergquist figgjo 咖啡馬克杯',
  'id': 'm10172014563',
  'img_url': 'https://u-mercari-images.mercdn.net/photos/m10172014563_1.jpg?w=200&h=200&fitcrop&sharpen',
  'name': '復古 Berquist Figgjo 咖啡馬克杯',
  'rerank_score': 0.0,
  'sparse_dist': None,
  'url': 'https://www.mercari.com/us/item/m10172014563'},
 {'dense_dist': 0.1960698664188385,
  'description': 'Pottery Barn 聖誕馴鹿馬克杯 - Dasher、Dancer、Prancer 和 Vixen 4 件組',
  'id': 'm81366738028',
  'img_url': 'https://u-mercari-images.mercdn.n

## 將工具新增到 **商店代理程式**

現在可以將工具新增到商店代理程式了。新增了以下部分：

- 在 `instruction` 中新增：`若要尋找項目，請使用 find_shopping_items 工具，傳遞一個查詢列表，並以項目的名稱、描述和圖片網址回覆使用者`
- 在 `Agent` 建構函式中新增 `tools` 參數：`tools=[find_shopping_items]`



In [ ]:
instruction = f'''
    您的角色是電子商務網站上的商店搜尋代理程式，該網站擁有數百萬件商品。您的職責是根據您收到的查詢搜尋商品。

    若要尋找項目，請使用 `find_shopping_items` 工具，傳遞一個查詢列表，並以項目的名稱、描述和圖片網址回覆使用者
'''

shop_agent = Agent(
    model='gemini-2.5-flash',
    name='shop_agent',
    description=(
        '電子商務網站的商店代理程式'
    ),
    instruction=instruction,
    tools=[find_shopping_items],
)

讓我們來測試這個代理程式。

In [27]:
await test_agent("有跳舞公仔的杯子", shop_agent)


>>> 使用者查詢：有跳舞公仔的杯子
-----
使用者查詢：['有跳舞公仔的杯子']
找到：3 個項目
-----
<<< 代理程式回應：我找到了三個符合您描述的項目：

*   **復活節裝飾跳舞太陽能商品 4 件組太陽能公仔：** 這是一套 4 件組太陽能復活節裝飾，以跳舞的公仔為特色。該套組包括一個在蛋中的跳舞兔子、一個在車裡的跳舞公兔、一個在車裡的跳舞母兔和一個跳舞的孵化小雞。它們高 4.5 英寸。
*   **Mayfair Collection By Jay-非洲女士/籃子/太陽-茶/咖啡杯/馬克杯：** 這個馬克杯以非洲女士、籃子和太陽為特色。馬克杯狀況非常好，沒有裂痕、缺口、裂紋或可見的刮痕。它高 4.25 英寸 x 寬 4.5 英寸（含把手）x 直徑 3.25 英寸。
*   **Hello Kitty 跳舞公仔：** 這是一個全新的 Hello Kitty 跳舞公仔。


## 定義具有 Google 搜尋 grounding 的 **研究代理程式**

接下來，我們將定義另一個代理程式 `research_agent`。此代理程式將接收使用者查詢，並使用內建的 Google 搜尋工具來研究人們針對使用者意圖購買哪種類型的商品。然後產生 5 個查詢以尋找這些商品。

請注意，以下代理程式定義將 `google_search` 指定為一個工具。這樣，代理程式就獲得了使用 Google 搜尋的能力。有關 Google 搜尋工具的詳細資訊，請參閱 ADK 文件中的 [Google 搜尋](https://google.github.io/adk-docs/tools/built-in-tools/#google-search)。

In [ ]:
from google.adk.tools import google_search

instruction = f'''
    您的角色是電子商務網站的市場研究員，該網站擁有數百萬件商品。

    當您收到使用者的搜尋請求時，請使用 Google 搜尋工具研究人們針對使用者意圖購買哪種類型的商品。

    然後，產生 5 個查詢以在電子商務網站上尋找這些商品並傳回它們。
'''

research_agent = Agent(
    model='gemini-2.5-flash',
    name='research_agent',
    description=('''
        電子商務網站的市場研究員。接收使用者的搜尋請求，並傳回 5 個產生的英文查詢列表。
    '''),
    instruction=instruction,
    tools=[google_search],
)

讓我們來測試這個代理程式。

In [30]:
await test_agent("給 10 歲男孩的生日禮物", research_agent)


>>> 使用者查詢：給 10 歲男孩的生日禮物
<<< 代理程式回應：好的，我可以幫您集思廣益一些給 10 歲男孩的禮物點子，然後為電子商務網站建立搜尋查詢。根據該年齡層目前的趨勢和熱門興趣，這裡有一些一般類別：

1.  **科技禮物/小工具：** 這可能包括耳機、智慧手錶或初學者友善的編碼套件等。
2.  **戶外/運動器材：** 像新籃球、滑板或露營裝備等物品通常很受歡迎。
3.  **建築/建構玩具：** 樂高積木組、模型套件或遙控車提供引人入勝的活動。
4.  **書籍/遊戲：** 考慮適合年齡的書籍、桌遊或電玩遊戲。
5.  **美術與工藝：** 如果男孩有創意，美術用品或 DIY 套件可能是不錯的選擇。

現在，讓我們根據這些類別為電子商務網站建立 5 個具體的搜尋查詢：




## 完成 **商店代理程式**

最後，我們修改 `shop_agent` 以使用 `reseach_agent` 和 `find_shopping_items` 工具。

In [ ]:
instruction = f'''
    您的角色是電子商務網站的購物禮賓員，該網站擁有數百萬件商品。請遵循以下步驟。

    當您收到使用者的搜尋請求時，請將其傳遞給 `research_agent` 工具，並接收 5 個產生的查詢。然後，將查詢列表傳遞給 `find_shopping_items` 以尋找商品。當您從工具收到商品列表時，請以商品的名稱、描述和圖片網址回覆使用者。
'''

shop_agent = Agent(
    model='gemini-2.5-flash',
    name='shop_agent',
    description=(
        '電子商務網站的購物禮賓員'
    ),
    instruction=instruction,
    tools=[
        AgentTool(agent=research_agent),
        find_shopping_items,
    ],
)

讓我們來測試這個代理程式。首先，使用者要求代理程式尋找項目。商店代理程式將呼叫研究代理程式，以使用 Google 搜尋結果產生查詢。

In [43]:
await test_agent("你能幫我找一個給我 10 歲兒子的生日禮物嗎？", shop_agent)


>>> 使用者查詢：你能幫我找一個給我 10 歲兒子的生日禮物嗎？
-----
使用者查詢：['給 10 歲男孩的 STEM 建築套件', '10 歲兒童的遙控車', '10 歲兒童的策略桌遊', '兒童初學者編碼遊戲', '10 歲兒童的戶外探險套件']
找到：15 個項目
-----
<<< 代理程式回應：這裡有一些給您 10 歲兒子的生日禮物點子：

*   **STEM 玩具：** 5 歲男孩的兒童玩具 - 5 合 1 拆裝男孩玩具 6-8 歲 STEM 玩具 5+ 歲男孩卡車變形為機器人建築玩具男孩禮物 5 6 7 歲男孩生日禮物。
    [https://u-mercari-images.mercdn.net/photos/m72066092335\_1.jpg?w=200&h=200&fitcrop&sharpen](https://u-mercari-images.mercdn.net/photos/m72066092335_1.jpg?w=200&h=200&fitcrop&sharpen)
*   **Lucky Doug 建築玩具模型飛機組：** 此建築玩具組包括 258 個零件和組件，例如螺母、螺栓、小螺絲起子和扳手等，由金屬製成，可承受粗暴玩耍而無需擔心容易倒塌。一個值得多花一點錢的精心製作的 STEM 建築套件，與全塑膠版本相比。此模型飛機高 4 英寸，長 13.8 英寸，寬 15.8 英寸。
    [https://u-mercari-images.mercdn.net/photos/m87990215453\_1.jpg?w=200&h=200&fitcrop&sharpen](https://u-mercari-images.mercdn.net/photos/m87990215453_1.jpg?w=200&h=200&fitcrop&sharpen)
*   **樂高：** 8-12 歲男孩的 STEM 建築玩具，6-8 歲男孩的組立套件，教育性建立機器人卡車套件。開箱/全新。
    [https://u-mercari-images.mercdn.net/photos/m81806559165\_1.jpg?w=200&h=200&fitcrop&sharpen](https://u-mercari-images.me

## 摘要

在本教學中，我們介紹了為電子商務平台建構多代理程式系統的過程，重點是「生成式推薦」。

透過這個過程，本教學說明了如何使用 ADK 和外部搜尋功能，建構一個能夠理解使用者意圖、執行研究、產生目標查詢，並在電子商務情境中提供相關產品推薦的複雜 AI 代理程式系統。
